In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os


load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEYY")



In [ ]:
from typing import TypedDict

from langchain.chat_models import init_chat_model
from langgraph.graph import START, StateGraph

joke_model =ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GEMINI_API_KEY
).with_config({"tags": ["joke"]})

poem_model = llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GEMINI_API_KEY
).with_config({"tags": ["poem"]})

# The tags are not tied to different APIs. They are just labels to distinguish outputs during streaming.


class State(TypedDict):
      topic: str
      joke: str
      poem: str


async def call_model(state, config):
      topic = state["topic"]
      print("Writing joke...")
      # Note: Passing the config through explicitly is required for python < 3.11
      # Since context var support wasn't added before then: https://docs.python.org/3/library/asyncio-task.html#creating-tasks
      
      joke_response = await joke_model.ainvoke(
            [{"role": "user", "content": f"Write a joke about {topic}"}],
            config, 
      )
      print("\n\nWriting poem...")
      poem_response = await poem_model.ainvoke(
            [{"role": "user", "content": f"Write a short poem about {topic}"}],
            config, 
      )
      return {"joke": joke_response.content, "poem": poem_response.content}


graph = (
      StateGraph(State)
      .add_node(call_model)
      .add_edge(START, "call_model")
      .compile()
)



In [4]:
async for msg, metadata in graph.astream(
      {"topic": "cats"},
      stream_mode="messages", 
):
    if metadata["tags"] == ["joke"]: 
        print(msg.content, end="|", flush=True)

Writing joke...
Why| was the cat sitting on the computer?  To keep an eye on the mouse|!
|

Writing poem...


# ✅ Why is async used in this code?
so you said that it is used when in one node mulltiple llms calls are to be called.
🧠 Why?
If you don’t use async, then:

Joke LLM will finish

Then Poem LLM will start

= ⏳ Slower

With async and await:

Both run at the same time

= ⚡ Faster response

# so let say if we have many nodes in grpah.
and each graph has one llms call(and response needed to stream to user from that specific node).
so in this case  i dont need async ?
am i right? and why 

❗Why "Filter by LLM Invocation" is Still Useful
Even though nodes run one-by-one:

Streaming happens token-by-token inside each node

If a node itself uses multiple LLMs or tools in parallel (with async)

Or if you want to build a real-time UI where you stream only part of the graph output

Or if you're debugging logs and only care about one step

Or you have shared components (like same LLM used in multiple places), tagging helps isolate them

SyntaxError: invalid character '❗' (U+2757) (3280518969.py, line 1)